This notebook performs feature engineering and model training.



Aggregating daily data to a quarterly level and creating new features (such as spend_per_user) improved the quality of inputs to the model.


This approach reduces noise in the data and allows for more robust analysis of trends at the quarterly level.


The normalization of the indices allows a more effective comparison between different quarters.

The linear regression was picked above other methods because of the small amount of data and also to avoid overfitting


Import and setup

In [5]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_percentage_error
import joblib

# Define paths
cleaned_data_path = "../data/processed/"
features_output_path = "../data/processed/"
model_output_path = "../models/"
os.makedirs(model_output_path, exist_ok=True)



In [6]:
def aggregate_to_quarterly(orders, transactions):
    orders["quarter"] = orders["date"].dt.to_period("Q")
    transactions["quarter"] = transactions["date"].dt.to_period("Q")

    orders_agg = orders.groupby("quarter")["order_number"].count().reset_index()
    orders_agg.rename(columns={"order_number": "total_orders"}, inplace=True)

    transactions_agg = transactions.groupby("quarter").agg({
        "total_spend_index": "mean",
        "weekly_active_users_index": "mean"
    }).reset_index()

    quarterly_data = pd.merge(orders_agg, transactions_agg, on="quarter", how="left")
    return quarterly_data

def normalize_features(quarterly_data):
    quarterly_data["spend_per_user"] = (
        quarterly_data["total_spend_index"] / quarterly_data["weekly_active_users_index"]
    )
    return quarterly_data


In [9]:
orders_clean = pd.read_csv(os.path.join(cleaned_data_path, "orders_cleaned.csv"))
transactions_clean = pd.read_csv(os.path.join(cleaned_data_path, "transactions_cleaned.csv"))

orders_clean["date"] = pd.to_datetime(orders_clean["date"])
transactions_clean["date"] = pd.to_datetime(transactions_clean["date"])

quarterly_data = aggregate_to_quarterly(orders_clean, transactions_clean)
quarterly_data = normalize_features(quarterly_data)

# Save features
quarterly_data.to_csv(os.path.join(features_output_path, "features.csv"), index=False)
quarterly_data.head()



,quarter,total_orders,total_spend_index,weekly_active_users_index,spend_per_user
0,2018Q1,12,1.020328,2.212705,0.461122
1,2018Q2,25,3.907097,5.081092,0.768948
2,2018Q3,23,9.063225,12.991272,0.697640
3,2018Q4,37,15.179657,19.689239,0.770962
4,2019Q1,25,13.636270,25.006123,0.545317


In [11]:
features = pd.read_csv(os.path.join(features_output_path, "features.csv"))
revenue = pd.read_csv(os.path.join(cleaned_data_path, "reported_cleaned.csv"))

# Align columns (remove spaces)
features["quarter"] = features["quarter"].astype(str).str.replace(" ", "", regex=False)
revenue["period"] = revenue["period"].str.replace(" ", "", regex=False)

df = pd.merge(features, revenue, left_on="quarter", right_on="period", how="inner")
df.drop(columns=["quarter", "start_date", "end_date"], errors="ignore", inplace=True)

df.head()



,total_orders,total_spend_index,weekly_active_users_index,spend_per_user,period,revenue_index
0,12,1.020328,2.212705,0.461122,2018Q1,100.000000
1,25,3.907097,5.081092,0.768948,2018Q2,253.521127
2,23,9.063225,12.991272,0.697640,2018Q3,139.788732
3,37,15.179657,19.689239,0.770962,2018Q4,306.161972
4,25,13.636270,25.006123,0.545317,2019Q1,117.693662


A linear regression model was used to predict the revenue_index.


The model was trained using a set of features derived from historical data, showing a good fit on the training set with a positive coefficient of determination (R²).


The model was validated using a specific quarter (2022Q4) to avoid data leakage problems.


In [12]:
#isolate 2022Q4 as test set, rest as train set
test = df[df["period"] == "2022Q4"]
train = df[df["period"] != "2022Q4"]

X_train = train.drop(columns=["revenue_index", "period"])
y_train = train["revenue_index"]

X_test = test.drop(columns=["revenue_index", "period"])
y_test = test["revenue_index"]

model = LinearRegression()
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

r2_train = r2_score(y_train, y_pred_train)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)
print(f"R² (train): {r2_train:.4f}")
print(f"MAPE (test): {mape_test:.4%}")

joblib.dump(model, os.path.join(model_output_path, "revenue_model.pkl"))
print("Model saved successfully.")


R² (train): 0.7943
MAPE (test): 19.7644%
Model saved successfully.
